# Planetary Motion

This little simulation was inspired by hearing that if we want to travel to mars, there is a special window every 2 years or so that makes it easier. So I wanted to understand why that is. One could use the orbital ellipses and simply let the planets follow them to simulate how the planets move in the solar system. But I was reading Newtons Principia and wanted to use Newton's Law of Universal Gravitation for my simulation.

Estimates of planet data is available at [NASA](https://nssdc.gsfc.nasa.gov/planetary/planetfact.html). However, for someone not fluent in astronomy, it can be quite tricky to interpret those fact sheets. For example; what does the angle 'Longitude of perihelion' represent? Turns out its a sum of two different angles that point in different directions... also perihelion is just a special word for periapsis when the thing it rotates around happens to be a star. The following picture from wikipedia greatly helped in interpreting the fact sheets.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Orbit1.svg/800px-Orbit1.svg.png" width="600" height="400">

I decided to use the following variables to construct a position and velocity of all planets when they are at their closest distance from the sun. It seems like an aesthetically pleasing starting point for a simulation.

1. (m) Mass
2. (pd) Periapsis distance (listed as perihelion distance)
3. (pv) Periapsis velocity (listed as Maximum Velocity)
4. (incl) Orbital inclination
5. (loan) Longitude of ascending node
6. (lop) Longitude of periapsis (listed as longitude of perihelion)

(aop) Argument of periapsis = Longitude of periapsis - Longitude of ascending node

In [1]:
function angles2rotm(a, b, c)
    #construct a rotation matrix from orbital elements (loan, incl, aop)
    # In particular, this is what by convention is called the Z(a) X(b) Z(c) rotation matrix
    # because it rotates around z-axis first, then around the x-axis, then around the z-axis again.
    # each time bringing the coordinate system with it just like in the picture above.
    R=zeros(3,3)
    R[:,1] = [cos(a)*cos(c) - sin(a)*cos(b)*sin(c), -cos(a)*sin(c) - sin(a)*cos(b)*cos(c), sin(a)*sin(b)]
    R[:,2] = [sin(a)*cos(c) + cos(a)*cos(b)*sin(c), -sin(a)*sin(c) + cos(a)*cos(b)*cos(c), -cos(a)*sin(b)]
    R[:,3] = [sin(b)*sin(c), sin(b)*cos(c), cos(b)]
    return R
end

function place_planets_at_periapsis(pd, pv, loan, incl, aop)
    p = zeros(9,3)
    v = zeros(9,3)
    for n=1:9
        R = angles2rotm(loan[n], incl[n], aop[n])
        p[n,:] = R*[pd[n],0,0]
        v[n,:] = R*[0,pv[n],0]
    end
    return p,v
end

place_planets_at_periapsis (generic function with 1 method)

In [2]:
function gravityforce(m, p, b, a)
    d = p[b,:]-p[a,:]
    F = -d .* 6.674e-11*m[a]*m[b]*(d'*d)^(-3/2)
    return F
end

function simulate_solar_system(p, v, m, Ndays) 
    F = zeros(9,3)
    Δt = 24*60*60
    dist_earth_to_planet = zeros(Ndays, 9)

    for t=1:Ndays
        for planet=2:9
            F[planet,:] = gravityforce(m, p, planet, 1)
        end
        a=F./m
        v += a.*Δt
        p += v.*Δt
        
        for planet=1:9
            dist_earth_to_planet[t, planet] = norm(p[planet,:]-p[4,:])/1e9
        end
    end
    return dist_earth_to_planet
end

simulate_solar_system (generic function with 1 method)

In [3]:
m = [1.9885e30, 0.33011e24, 4.8675e24, 5.9724e24, 0.64171e24, 1898.19e24, 568.34e24, 86.813e24, 102.413e24]
pd = [0, 46.00e9, 107.48e9, 147.09e9, 206.62e9, 740.52e9, 1352.55e9, 2741.30e9, 4444.45e9]
pv = [0, 58.98e3, 35.26e3, 30.29e3, 26.50e3, 13.72e3, 10.18e3, 7.11e3, 5.50e3]
incl = π/180*[0, 7.00487, 3.39471, 0.00005, 1.85061, 1.30530, 2.48446, 0.76986, 1.76917]
loan = π/180*[0, 48.33167, 76.68069, -11.26064, 49.57854, 100.55615, 113.71504, 74.22988, 131.72169]
lop  = π/180*[0, 77.45645, 131.53298, 102.94719, 336.04084, 14.75385, 92.43194, 170.96424, 44.97135];
aop = lop - loan;

p, v = place_planets_at_periapsis(pd, pv, loan, incl, aop)
dist_earth_to_planet = simulate_solar_system(p, v, m, 365*100);

In [4]:
function avgperiod(e2p)
    binaray_toward_away = e2p[2:end].>e2p[1:end-1]
    index_of_minima = find(binaray_toward_away[2:end].>binaray_toward_away[1:end-1])
    #periods = index_of_minima[2:end].-index_of_minima[1:end-1]
    #avgperiod = mean(periods[2:end-1])
    meanperiod = mean(index_of_minima[2:end].-index_of_minima[1:end-1])
    return meanperiod
end

names=["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune"]
println("Average period of Earth close approach with")
for planet=[1,2,3,5,6,7,8,9]
    period = avgperiod(dist_earth_to_planet[:,planet])
    println(names[planet], ": ", Int(round(period)), " days")
    #println(names[planet], ": ", round(period,1), " days")
end

Average period of Earth close approach with
Sun: 365 days
Mercury: 117 days
Venus: 585 days
Mars: 780 days
Jupiter: 399 days
Saturn: 378 days
Uranus: 370 days
Neptune: 368 days


# Visualization

In [5]:
using Plots; gr(size=(600,750))
plot(leg=false, xtick=false, ytick=false, axis=false)

function simulate_solar_system_VISUALIZE(p, v, m, Ndays) 
    F = zeros(9,3)
    Δt = 24*60*60
    ss = norm(p[5,:])*1.5 #size of plot
    
    @gif for t=1:Ndays
        for n=2:9; F[n,:] = gravityforce(m, p, n, 1) end
        a=F./m
        v += a.*Δt
        p += v.*Δt

        # visualize
        if t%5==0
            p1 = scatter(p[:,1],p[:,2], xlim=(-ss,ss), ylim=(-ss,ss), leg=false, xtick=false, ytick=false, title="Inner Solar System", axis=false)
            plot!(p1, [p[4,1];p[5,1]], [p[4,2];p[5,2]]) # add a line between earth and mars
            p2 = plot(dist_earth_to_planet[1:t,5], ylim=(2e10/1e9, 5e11/1e9), title="Distance Earth-Mars (in million kilometers)", xlab="days")
            plot(p1,p2, layout=grid(2,1,heights=[0.8,0.2]), leg=false)
        end

    end every 5
end

p, v = place_planets_at_periapsis(pd, pv, loan, incl, aop)
simulate_solar_system_VISUALIZE(p, v, m, 365*10);

INFO: Saved animation to /home/andy/dev/githubnotebooks/earth2mars/tmp.gif


<img src="tmp.gif">

#### A note on the animation; although the simulatation is in 3 dimensions, the plot is looking straight down on the solarsystem making it look 2 dimensional. However, the plotted earth-mars distance is the correct distance in 3-dimensional space.

# Not all close encounters are equal
As seen in the animation above, there is some funky-ness going on in the earth-mars distance. This comes from the fact that the orbits are not perfectly circular and not perfectly aligned in the same place etc. We have already calculated that the average period of close encounters with mars is about 780 days. But some close encounters are closer than others. We can calculate unusually close... close encounters.

In [6]:
gr(size=(800,1600))
L=20000
p1=plot(dist_earth_to_planet[1:L,1], title=names[1])
p2=plot(dist_earth_to_planet[1:L,2], title=names[2])
p3=plot(dist_earth_to_planet[1:L,3], title=names[3])
p4=plot(dist_earth_to_planet[1:L,5], title=names[5])
p5=plot(dist_earth_to_planet[1:L,6], title=names[6])
p6=plot(dist_earth_to_planet[1:L,7], title=names[7])
p7=plot(dist_earth_to_planet[1:L,8], title=names[8])
p8=plot(dist_earth_to_planet[1:L,9], title=names[9])
#plot(p1,p2,p3,p4,p5,p6,p7,p8, layout=grid(8,1,heights=[0.1,0.1,0,1.0,1,0.1,0.1,0,1.0,1]), leg=false)
#plot(p1,p2,p3, layout=grid(3,1, heights=[0.1,0.1,0.1]), leg=false)
plot(p1,p2,p3,p4,p5,p6,p7,p8, layout=grid(8,1), leg=false)
#plot(p1,p2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
 
 147 
 
 
 148 
 
 
 149 
 
 
 150 
 
 
 151 
 
 
 152 
 
 
 Sun 
 
<polyline clip-path="url(#clip6202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 36.8348,164.412 36.872,164.943 36.9092,165.454 36.9464,165.947 36.9835,166.419 37.0207,166.872 37.0579,167.305 37.0951,167.717 37.1323,168.109 37.1694,168.481 
 37.2066,168.833 37.2438,169.164 37.281,169.474 37.3181,169.763 37.3553,170.031 37.3925,170.279 37.4297,170.505 37.4668,170.71 37.504,170.894 37.5412,171.056 
 37.5784,171.198 37.6155,171.317 37.6527,171.416 37.6899,171.493 37.7271,171.548 37.7642,171.582 37.8014,171.594 37.8386,171.585 37.8758,171.554 37.9129,171.502 
 37.9501,171.428 37.9873,171.333 38.0245,171.216 38.0616,171.078 38.0988,170.919 38.136,170.738 38.1732,170.536 38.2104,170.313 38.2475,170.069 38.2847,169.803 
 38.3219,169.517 38.3591,169.21 38.3962,168.882 38.4334,168.534 38.4706,168.165 38.5078,167.775 38.5449,167.366 38.5821,166.936 38.6193,166.486 38.6565,166.017 
 38.6936,165.527 38.7308,165.018 38.768,164.49 38.8052,163.943 38.8423,163.376 38.8795,162.791 38.9167,162.187 38.9539,161.565 38.991,160.924 39.0282,160.266 
 39.0654,159.589 39.1026,158.895 39.1397,158.184 39.1769,157.456 39.2141,156.71 39.2513,155.948 39.2885,155.17 39.3256,154.375 39.3628,153.565 39.4,152.739 
 39.4372,151.897 39.4743,151.04 39.5115,150.169 39.5487,149.283 39.5859,148.383 39.623,147.468 39.6602,146.54 39.6974,145.599 39.7346,144.644 39.7717,143.677 
 39.8089,142.697 39.8461,141.705 39.8833,140.702 39.9204,139.686 39.9576,138.66 39.9948,137.622 40.032,136.574 40.0691,135.516 40.1063,134.448 40.1435,133.37 
 40.1807,132.283 40.2178,131.187 40.255,130.082 40.2922,128.969 40.3294,127.849 40.3666,126.72 40.4037,125.585 40.4409,124.443 40.4781,123.294 40.5153,122.139 
 40.5524,120.978 40.5896,119.812 40.6268,118.641 40.664,117.465 40.7011,116.284 40.7383,115.1 40.7755,113.912 40.8127,112.721 40.8498,111.527 40.887,110.33 
 40.9242,109.131 40.9614,107.93 40.9985,106.728 41.0357,105.524 41.0729,104.32 41.1101,103.115 41.1472,101.91 41.1844,100.706 41.2216,99.5018 41.2588,98.299 
 41.296,97.0975 41.3331,95.8977 41.3703,94.6999 41.4075,93.5046 41.4447,92.312 41.4818,91.1225 41.519,89.9365 41.5562,88.7543 41.5934,87.5762 41.6305,86.4026 
 41.6677,85.2338 41.7049,84.0701 41.7421,82.9119 41.7792,81.7596 41.8164,80.6134 41.8536,79.4737 41.8908,78.3407 41.9279,77.2149 41.9651,76.0965 42.0023,74.9859 
 42.0395,73.8834 42.0766,72.7892 42.1138,71.7038 42.151,70.6273 42.1882,69.5602 42.2253,68.5026 42.2625,67.455 42.2997,66.4175 42.3369,65.3905 42.3741,64.3743 
 42.4112,63.3691 42.4484,62.3753 42.4856,61.393 42.5228,60.4227 42.5599,59.4644 42.5971,58.5186 42.6343,57.5855 42.6715,56.6652 42.7086,55.7582 42.7458,54.8646 
 42.783,53.9846 42.8202,53.1186 42.8573,52.2667 42.8945,51.4292 42.9317,50.6062 42.9689,49.7982 43.006,49.0051 43.0432,48.2274 43.0804,47.4651 43.1176,46.7185 
 43.1547,45.9878 43.1919,45.2732 43.2291,44.5748 43.2663,43.8929 43.3034,43.2277 43.3406,42.5792 43.3778,41.9478 43.415,41.3336 43.4522,40.7366 43.4893,40.1572 
 43.5265,39.5954 43.5637,39.0514 43.6009,38.5254 43.638,38.0174 43.6752,37.5277 43.7124,37.0563 43.7496,36.6033 43.7867,36.169 43.8239,35.7534 43.8611,35.3565 
 43.8983,34.9787 43.9354,34.6198 43.9726,34.28 44.0098,33.9595 44.047,33.6583 44.0841,33.3764 44.1213,33.114 44.1585,32.8711 44.1957,32.6477 44.2328,32.4441 
 44.27,32.2601 44.3072,32.0958 44.3444,31.9514 44.3815,31.8267 44.4187,31.7219 44.4559,31.637 44.4931,31.572 44.5303,31.527 44.5674,31.5018 44.6046,31.4966 
 44.6418,31.5114 44.679,31.5461 44.7161,31.6007 44.7533,31.6752 44.7905,31.7697 44.8277,31.884 44.8648,32.0181 44.902,32.1721 44.9392,32.3458 44.9764,32.5392 
 45.0135,32.7523 45.0507,32.985 45.0879,33.2373 45.1251,33.509 45.1622,33.8002 45.1994,34.1107 45.2366,34.4405 45.2738,34.7894 45.3109,35.15

In [7]:
p, v = place_planets_at_periapsis(pd, pv, loan, incl, aop)
dist_earth_to_planet = simulate_solar_system(p, v, m, 1000000) #run longer to capture unusually close encounters with outer planets

function avgperiod2(e2p)
    binary_toward_away = e2p[2:end].>e2p[1:end-1]
    index_of_minima = find(binary_toward_away[2:end].>binary_toward_away[1:end-1])
    dist_of_minima=e2p[index_of_minima]

    binary_toward_away2 = dist_of_minima[2:end].>dist_of_minima[1:end-1]
    index_of_minima2 = find(binary_toward_away2[2:end].>binary_toward_away2[1:end-1])

    acc_index = index_of_minima[index_of_minima2]
    meanperiod2 = mean(acc_index[2:end]-acc_index[1:end-1])
end

println("Average period of Earth UNUSUALLY close approach with")
for planet=[2,3,5,6,7,8,9]
    period = avgperiod2(dist_earth_to_planet[:,planet])
    println(names[planet], ": ", Int(round(period)), " days")
    #println(names[planet], ": ", round(period,1), " days")
end

Average period of Earth UNUSUALLY close approach with
Mercury: 363 days
Venus: 1463 days
Mars: 5699 days
Jupiter: 4346 days
Saturn: 10830 days
Uranus: 30658 days
Neptune: 60365 days


# Conclustions
If we want to get on a spaceship and travel to Mars, there is a window approximately every 2.14 years when mars is close. In addition to that, there is an unusually good window every 15.6 years.

//Anders